<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Run Notebook on AML
This noteboook provides an example of how to directly submit notebook to AzureML compute targets. 

# 0 Setup environment

### Install azure.contrib.notebook package

In [1]:
!pip install "azureml-contrib-notebook>=1.0.21.2"

In [2]:
#%run ../../scripts/generate_conda_file.py

### Set up AzureML workspace
Please refer to [configuration.ipynb](https://github.com/Azure/MachineLearningNotebooks/blob/master/configuration.ipynb) to set up Azure account, subscription, workspace and compute target. Below code assumes config.json file is in place.

In [3]:
import os
from azureml.core import Workspace
from os import path, makedirs
import shutil
from azureml.core import Experiment
from azureml.core.runconfig import RunConfiguration
from azureml.contrib.notebook.notebook_run_config import NotebookRunConfig, PapermillExecutionHandler
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core.conda_dependencies import CondaDependencies

subscription_id = os.getenv("SUBSCRIPTION_ID", default="<my-subscription-id>")
resource_group = os.getenv("RESOURCE_GROUP", default="<my-resource-group>")
workspace_name = os.getenv("WORKSPACE_NAME", default="<my-workspace-name>")
workspace_region = os.getenv("WORKSPACE_REGION", default="eastus2")

ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)

This example uses run-based AzureML Compute. Alternatively, you can use persistent compute target (cpu or gpu cluster) or remote virtual machines. For more details, see [How to set up training targets](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets). The VM size (`STANDARD_D2_V2`) used here costs $0.114/hour.

In [8]:
NOTEBOOK_NAME = 'sar_movielens.ipynb'
EXPERIMENT_NAME = 'sdk-papermill'
exp_directory = 'test_dir' + EXPERIMENT_NAME
# create experiment directory
if not path.exists(exp_directory):
    makedirs(exp_directory)
notebook_path = path.join(exp_directory, NOTEBOOK_NAME)
shutil.copy(NOTEBOOK_NAME, notebook_path)
        
exp = Experiment(workspace=ws, name=EXPERIMENT_NAME)

run_config_aml = RunConfiguration()
run_config_aml.target = "amlcompute" # possible to use different compute targets
run_config_aml.amlcompute.vm_size = 'STANDARD_D2_V2'
run_config_aml.environment.docker.enabled = True
run_config_aml.environment.docker.base_image = DEFAULT_CPU_IMAGE
run_config_aml.environment.python.user_managed_dependencies = False
run_config_aml.auto_prepare_environment = True
run_config_aml.environment.python.conda_dependencies = CondaDependencies(conda_dependencies_file_path='../../myenv2.yaml')
#run_config_aml.environment.python.conda_dependencies = CondaDependencies(conda_dependencies_file_path='../../reco_base.yaml')

handler = PapermillExecutionHandler(kernel_name='python3')

cfg = NotebookRunConfig(source_directory='../../',
                                notebook='notebooks/00_quick_start/' + NOTEBOOK_NAME,
                                output_notebook='outputs/out.ipynb',
                                parameters={"MOVIELENS_DATA_SIZE": "100k", "TOP_K": 10},
                                run_config=run_config_aml)
run = exp.submit(cfg)
run

ExperimentExecutionException: {
    "error_details": {
        "correlation": {
            "operation": "fbe475d5444d1b4aa764eedf64239b8a",
            "request": "+Jr+XDz73HA="
        },
        "error": {
            "code": "UserError",
            "debugInfo": {
                "message": "Specified base docker image mcr.microsoft.com/azureml/base:0.2.4 not found",
                "stackTrace": "   at Microsoft.MachineLearning.EnvironmentManagement.Services.DerivedImageManager.GetSpecification(EnvironmentDefinition environmentDefinition) in /home/vsts/work/1/s/src/azureml-api/src/EnvironmentManagement/Services/DerivedImageManager.cs:line 46\n   at Microsoft.MachineLearning.EnvironmentManagement.EntryPoints.Api.Controllers.CloudMaterializationController.GetImageDetails(Guid subscriptionId, String resourceGroupName, String workspaceName, String environmentName, Nullable`1 environmentVersion) in /home/vsts/work/1/s/src/azureml-api/src/EnvironmentManagement/EntryPoints/Api/Controller

You should see simialar output as below
![Experiment submit output](https://docs.microsoft.com/en-us/azure/machine-learning/service/media/quickstart-get-started/view_exp.png)
Azure Portal looks like this
![Azure Portal Experiment](https://docs.microsoft.com/en-us/azure/machine-learning/service/media/quickstart-get-started/web-results.png)

In [ ]:
run.wait_for_completion(show_output=True)